In [1]:
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pandas as pd

In [2]:
conf = (SparkConf()
            .setAppName('data_explore')
            .setMaster('spark://spark-master:7077')
       )
conf.set("spark.executor.memory", "6g")
conf.set("spark.driver.maxResultSize", "0")
conf.set("spark.sql.shuffle.partitions", "10")

In [3]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [4]:
df = (spark.read.format('csv')
      .option('inferSchema', 'true')
      .option('escape', '"')
      .option('header', 'true')
      .load('hdfs://namenode:9000/data/process_logs.csv')
     )

## Data Cleaning and feature engineering
+ parse process name/process path
+ split `Process_Command_Line` into tokens to count arguments
+ add boolean value to identify if process was started by an administrator
+ add boolean value to identify if path is in trusted directory: [C:\Program Files, C:\Windows]

In [23]:
from pyspark.sql.functions import regexp_extract, regexp_replace, col, count, split, size, to_date
clean_df = df.select(
    '*',
    regexp_extract(col('New_Process_Name'), r'(\w+.?\w+$)', 0).alias('process_name'),
    regexp_extract(col('New_Process_Name'), r"(\w?:?.*\\?\w+\s?\w+\\)", 0).alias('process_path'),
    regexp_replace(col('user'), r'\$', '').alias('user_name'),
    split(col('Process_Command_Line'), '\s+').alias('cmd_line_split')
).selectExpr(
        'Account_Domain as domain', 'Creator_Process_ID as creator_process_id', 'EventCode as event_code', 'host', 'Logon_ID as logon_id', 'member_id', \
        'New_Process_ID as new_process_id', 'Process_Command_Line as command_line', 'process_name', 'process_path', 'session_id', 'user_name', 'cmd_line_split', '_time as time'
).withColumn(
    'is_admin', col('member_id').isin(['FROTHLY\\administrator', 'NT AUTHORITY\\SYSTEM', 'NT AUTHORITY\\LOCAL SERVICE', 'NT AUTHORITY\\NETWORK SERVICE']).cast('int')
).withColumn(
    'trusted_path', (col('process_path').like('%C:\Program Files%') | col('process_path').like('%C:\Windows%')).cast('int')
).dropna(how='any')


In [24]:
clean_df.groupBy('user_name','is_admin','process_name','trusted_path','process_path').count().show(truncate=False)

+------------------+--------+----------------------+------------+------------------------------------------------------------------------------------+-----+
|user_name         |is_admin|process_name          |trusted_path|process_path                                                                        |count|
+------------------+--------+----------------------+------------+------------------------------------------------------------------------------------+-----+
|fyodor.malteskesko|0       |chrome.exe            |1           |C:\Program Files (x86)\Google\Chrome\Application\                                   |2109 |
|WRK-BTUN          |1       |aitagent.exe          |1           |C:\Windows\System32\                                                                |2    |
|VENUS             |1       |DWHWizrd.exe          |1           |C:\Program Files (x86)\Symantec\Symantec Endpoint Protection\12.1.7266.6800.105\Bin\|1    |
|VENUS             |1       |netmon.exe            |1     

## process count

In [25]:
from pyspark.sql.functions import count, desc, asc, expr
df_process_count = clean_df.groupBy('process_name').agg(
    count('process_name').alias('process_count')
)

In [26]:
df_process_count.orderBy(desc('process_count')).show(20, False)

+----------------------+-------------+
|process_name          |process_count|
+----------------------+-------------+
|powershell.exe        |124310       |
|winprintmon.exe       |74964        |
|MonitorNoHandle.exe   |70836        |
|netmon.exe            |68734        |
|winhostinfo.exe       |66849        |
|admon.exe             |55794        |
|conhost.exe           |39452        |
|cscript.exe           |27535        |
|dllhost.exe           |13691        |
|schtasks.exe          |9187         |
|chrome.exe            |8744         |
|cmd.exe               |7915         |
|wsmprovhost.exe       |4901         |
|SearchProtocolHost.exe|4862         |
|slui.exe              |4438         |
|SearchFilterHost.exe  |3091         |
|GoogleUpdate.exe      |2811         |
|WmiPrvSE.exe          |2582         |
|taskhost.exe          |2372         |
|OSPPSVC.EXE           |1722         |
+----------------------+-------------+
only showing top 20 rows



# create additional features
+ number of command line arguments
+ count of path 
+ if executable is common attack tool (i.e. at.exe, sc.exe, powershell.exe)

## command line argument count

In [27]:
from pyspark.sql.functions import split
df_arg_count = clean_df.select(
    '*'
).selectExpr(
    'command_line',
    'size(cmd_line_split) as argument_count'
)

In [28]:
df_arg_count.show(10, False)

+---------------------------------------------------------------------------+--------------+
|command_line                                                               |argument_count|
+---------------------------------------------------------------------------+--------------+
|"C:\Program Files\SplunkUniversalForwarder\bin\splunk-netmon.exe"          |2             |
|"C:\Program Files\SplunkUniversalForwarder\bin\splunk-powershell.exe" --ps2|3             |
|"C:\Program Files\SplunkUniversalForwarder\bin\splunk-netmon.exe"          |2             |
|"C:\Program Files\SplunkUniversalForwarder\bin\splunk-winhostinfo.exe"     |2             |
|"C:\Program Files\SplunkUniversalForwarder\bin\splunk-powershell.exe" --ps2|3             |
|"C:\Program Files\SplunkUniversalForwarder\bin\splunk-powershell.exe"      |2             |
|"C:\Program Files\SplunkUniversalForwarder\bin\splunk-netmon.exe"          |2             |
|"C:\Program Files\SplunkUniversalForwarder\bin\splunk-winhostinfo.exe

## path count
+ count the number of times of the executable's path 

In [29]:
df_path_count = clean_df.select('process_path').groupBy('process_path').count().orderBy(desc('count'))

In [30]:
df_path_count.show(5, False)

+-------------------------------------------------+------+
|process_path                                     |count |
+-------------------------------------------------+------+
|C:\Program Files\SplunkUniversalForwarder\bin\   |456891|
|C:\Windows\System32\                             |110607|
|C:\Windows\SysWOW64\                             |17233 |
|C:\Program Files (x86)\Google\Chrome\Application\|8744  |
|C:\Windows\System32\WindowsPowerShell\           |4628  |
+-------------------------------------------------+------+
only showing top 5 rows



## common_attack_tool check
+ import splunk tools.csv and create tool array
+ check to see if the exectuble is a common attack tools such as [at.exe, sc.exe, powershell.exe]

In [31]:
tool_df = (spark.read.format('csv')
           .option('inferschema', 'true')
           .option('header', 'true')
           .load('hdfs://namenode:9000/data/tools.csv')
          )

In [32]:
# create list of tool names by taking the value of the filename key from every row object in tool_df
tool_list = [x['filename'] for x in tool_df.select('filename').collect()]

In [33]:
util_df = clean_df.groupBy('process_name').agg(
    clean_df['process_name'].isin(tool_list).cast('int').alias('common_attack_tool')
)

In [34]:
util_df.show()

+--------------------+------------------+
|        process_name|common_attack_tool|
+--------------------+------------------+
|         OSPPSVC.EXE|                 0|
|          wermgr.exe|                 0|
|         cscript.exe|                 0|
|             tor.exe|                 0|
|VMwareResolutionS...|                 0|
|          cvtres.exe|                 0|
|       SymCorpUI.exe|                 0|
|          tlsbln.exe|                 0|
|         ehsched.exe|                 0|
|     vncserverui.exe|                 0|
|             mmc.exe|                 0|
|      taskhostex.exe|                 0|
|        regsvr32.exe|                 0|
|        ccSvcHst.exe|                 0|
|             dwm.exe|                 0|
|         perfmon.exe|                 0|
|OfficeClickToRun.exe|                 0|
|     winprintmon.exe|                 0|
|             wmi.exe|                 0|
|        DVDMaker.exe|                 0|
+--------------------+------------

# combine data frames

In [35]:
df_process = clean_df.join(df_process_count, on='process_name').selectExpr(
    'process_name', 'process_count', 'command_line', 'process_path', 'is_admin', 'trusted_path'
)

In [36]:
df_arg = df_process.join(df_arg_count, on='command_line').selectExpr(
    'process_name', 'process_count', 'command_line', 'process_path', 'is_admin', 'trusted_path', 'argument_count'
)

In [37]:
df_arg.show(5)

+--------------------+-------------+--------------------+--------------------+--------+------------+--------------+
|        process_name|process_count|        command_line|        process_path|is_admin|trusted_path|argument_count|
+--------------------+-------------+--------------------+--------------------+--------+------------+--------------+
|SearchProtocolHos...|         4862|"C:\Windows\syste...|C:\Windows\System32\|       1|           1|            19|
|SearchProtocolHos...|         4862|"C:\Windows\syste...|C:\Windows\System32\|       1|           1|            19|
|SearchProtocolHos...|         4862|"C:\Windows\syste...|C:\Windows\System32\|       1|           1|            19|
|SearchProtocolHos...|         4862|"C:\Windows\syste...|C:\Windows\System32\|       1|           1|            19|
|SearchProtocolHos...|         4862|"C:\Windows\syste...|C:\Windows\System32\|       1|           1|            19|
+--------------------+-------------+--------------------+---------------

In [ ]:
df_path = df_arg.join(df_path_count, on='process_path').selectExpr(
    'process_name', 'process_count', 'command_line', 'process_path', 'is_admin', 'trusted_path', 'argument_count', 'count'
).show(10,False) 